# BRFSS Playground - HBR

Heiarchical Bayesian Regression to Predict Mental Health

In [115]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import halfcauchy
from sklearn.metrics import f1_score
from scipy.optimize import minimize

In [2]:
brfss_norm = pd.read_csv('../data/brfss_normed.csv.gz')

In [3]:
brfss_norm.shape

(96986, 63)

### Small cleaning

In [52]:
drop_vars = ['B_ID', 'SMP_WGHT', 'MNTL_HLTH_LEV_BRFSS', 'MENTAL_HEALTH_30_BRFSS']
brfss_vars = [
    c for c in brfss_norm.columns.values if c not in drop_vars
]
X = brfss_norm[brfss_vars].copy()
y = brfss_norm['MNTL_HLTH_LEV_BRFSS'].copy()
y_simple = y.copy()
y_simple.loc[y_simple > 0] = 1
y_simple.loc[y_simple == 0] = 0

Split training/test

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
behaviors = [
    'ROUTINE_CHECK_BRFSS',
    'INTERNET_USE_BRFSS',
    'SMK_NOW_BRFSS',
    'TRY_QUIT_SMK_BRFSS',
    'SNUFF_BRFSS',
    'CNSM_FT_DAY_BRFSS',
    'PA_CAT_BRFSS',
    'AER_STRNGH_GUIDE_BRFSS',
    'PHYS_HLTH_LEV_BRFSS',
    'HVY_DRNKR_BRFSS',
    'AVG_NUM_DRNK_30_BRFSS',
    'BINGE_DRNK_30_BRFSS',
    'DRNK_PER_DAY_BRFSS',
    'DLY_FF_SERVE_BRFSS',
    'DLY_FT_SERVE_BRFSS',
    'DLY_FJ_SERVE_BRFSS',
    'DLY_GRN_VEG_SERVE_BRFSS',
    'LARGE_NUM_DRNK_30_BRFSS',
    'MET_VAL_BRFSS',
    'MET_VAL_OTHR_BRFSS',
    'TTL_MIN_OF_PA_WEEK_BRFSS',
    'TTL_MIN_OF_VIG_WEEK_BRFSS',
    'MIN_OF_PA_WEEK_BRFSS',
    'MIN_OF_PA_WEEK_OTHR_BRFSS',
    'MIN_OF_VIG_WEEK_BRFSS',
    'MIN_OF_VIG_WEEK_OTHR_BRFSS',
    'DLY_POTATO_SERVE_BRFSS',
    'DLY_OTHR_VEG_SERVE_BRFSS',
    'NUM_DRNKS_PER_WEEK_BRFSS',
    'TOTAL_FT_DAY_BRFSS',
    'MIN_OF_EX_WEEK_BRFSS',
    'MIN_OF_EX_WEEK_OTHR_BRFSS',
    'TOTAL_VEG_DAY_BRFSS'
]

## Write functions

In [74]:
def sample_weights(mu_weights, std_weights):
    """
    Sample a set of logistic regression weights for each person
    
    :param mu_weights: np.array<float>, num_clusters x num_weights matrix 
                       representing the mean of the distribution of the weights
    :param std_weights: np.array<float>, num_clusters x num_weights vector 
                       representing the std of the distribution of the weights
                       
    :return sampled_weights: np.array<float>, num_clusters x num_weights matrix
                             represents the sample weight for each cluster
    """
    # Setup matrices
    sampled_weights = np.zeros(shape=(mu_weights.shape[0], mu_weights.shape[1]))
    
    # Sample the weights
    for i in range(sampled_weights.shape[0]):
        for j in range(sampled_weights.shape[1]):
            # Sample the weight
            sampled_weights[i, j] = np.random.normal(loc=mu_weights[i, j], scale=std_weights[i, j])
            
    return sampled_weights

In [97]:
def sigmoid(x):
    """
    Calculate the sigmoid function.
    
    :param x: np.ndarray<float>, 1 x n vector of values
    
    :return: 1 / (1 + exp(-x))
    """
    return 1 / (1 + np.exp(-1 * x))
    
def calculate_result(weights, cluster_matrix, training_data, thresh):
    """
    Calculate the logistic regression value given a weights matrix and set of stats.
    
    :param weights: np.array<float>, num_clusters x num_weights matrix
                    the logistic regression weights for each cluster
    :param cluster_matrix: np.array<float>, num_people x num_clusters matrix
                    the matrix showing how much each individual (task) is within each cluster
    :param training_data: np.array<float>, num_people x num_variables matrix
                    the training data
    :param thresh: float, threshold for classfication
    """
    # Go through the training data, and get a calculation for each cluster
    y_prob = np.zeros(shape=(cluster_matrix.shape[0], 1))
    
    for i in range(training_data.shape[0]):
        # First get results per cluster
        person_result = np.sum(weights * np.transpose(training_data[i, :]), axis=1)
        # Send the vector through a sigmoid function
        person_result = sigmoid(person_result)
        # Now multiply by weight
        person_result = np.sum(cluster_matrix[i] * person_result)
        # Store
        y_prob[i] = person_result
        
    y_prob[y_prob > thresh] = 1
    y_prob[y_prob <= thresh] = 0
    
    return y_prob

In [116]:
def hblr(X_train, y_train, thresh, cluster_matrix, mu_weights, std_weights):
    """
    Solve the HBLR regression
    
    :param X_train: np.array<float>, n_people x n_params, the training data
    :param y_train: np.array<float>, n_people x 1, the training objective
    :pram thresh: <float>, threshold for classification
    :param cluster_matrix: np.array<float>, num_people x num_clusters matrix
                    the matrix showing how much each individual (task) is within each cluster
   
    :param mu_weights: np.array<float>, num_clusters x num_weights matrix 
                       representing the mean of the distribution of the weights
    :param std_weights: np.array<float>, num_clusters x num_weights vector 
                       representing the std of the distribution of the weights
                       
    :return f1_score: <float> The f1_score of the current model
    """
    
    # First get weights
    weights = sample_weights(mu_weights, std_weights)
    
    # Then run prediction
    y_pred = calculate_result(weights, cluster_matrix, X_train, thresh)
    
    # Return F1
    print(y_pred.shape)
    return -1 * f1_score(y_train, y_pred)

In [46]:
test_mu_normal_mu = np.array(
    [[0, 0], [0, 0]]
)

test_mu_normal_std = np.array(
    [[1, 1], [1, 1]]
)

test_std_cauchy_scale = np.array(
    [[1, 1], [1, 1]]
)

test_weights = sample_weights(test_mu_normal_mu, test_mu_normal_std, test_std_cauchy_scale)

In [57]:
test_weights = np.array([
    [0.3, 0.5, 0.1],
    [0.2, 0.8, 0.05],
    [0.1, 0.4, 0.6]
])

test_training_data = np.array([
    [1, 2, 3],
    [1, 4, 5]
])

test_cluster_matrix = np.array([
    [0.1, 0.4, 0.5],
    [0.3, 0.6, 0.1]
])

test_thresh = 0.4

calculate_result(test_weights, test_cluster_matrix, test_training_data, test_thresh)

array([[1.],
       [1.]])

Try it

In [103]:
X_samp = X.sample(n=5000, random_state=42)
X_samp = pd.concat([pd.DataFrame(data=np.ones(shape=(X_samp.shape[0], 1)), index=X_samp.index), X_samp], axis=1)
y_samp = y.loc[X_samp.index]
y_samp[y_samp > 0] = 1.0

In [108]:
n_clusters = 5
thresh_0 = 0.7
cluster_matrix_0 = np.ones(shape=(X_samp.shape[0], n_clusters)) * (1.0 / n_clusters)
mu_weights_0 = np.zeros(shape=(n_clusters, X_samp.shape[1]))
std_weights_0 = np.ones(shape=(n_clusters, X_samp.shape[1]))

In [114]:
hblr(X_samp.as_matrix(), y_samp.as_matrix(), thresh_0, cluster_matrix_0, mu_weights_0, std_weights_0)

(5000, 1)


/Users/daadler0309/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


0.10499227997941328

In [ ]:
minimize(
    fun=hblr,
)